# Loops and iteration

_Note_: ensure that students copy, by hand and on paper, the various definitions written by the teacher on the whiteboard. It is strongly advised to ask students *not* to use a laptop, as it will prove distracting.

## The issue
- The programs we have written so far can only perform a limited number of steps maximum
- `x := 0; y := 2; done` performs a maximum of two steps
- `x := int(input()); if x > 0 then { x := x * 2; x := x + 1 } else { x := -x; x := x * 2; x := x + 1 }` performs a maximum of four steps (discuss why?)
- let us draw a program execution in the graph of all possible states as a *finite sub-graph*
- not all interesting behaviours that we might want to encode as a program are *finite*
- let us begin with a simple example

```
l := int(input())
if l <= 0 then
  s := ""
elif l = 1 then
  s := "*"
elif l = 2 then
  s := "**"
elif l = 3 then
  s := "***"
...
else
  s := "too long, sorry :*("
```
- the program clearly tries to produce a string of asterisks that is as long as the input, but to properly do this we would need to write an infinite program
    - this is not really viable
    

## Loops
- let us observe the structure of the program above
- let us give a name to each of the nested sub-programs:
```
l := int(input())
if l <= 0 then
  s := ""               P0
elif l = 1 then
  s := "*"              P1
elif l = 2 then
  s := "**"             P2
elif l = 3 then
  s := "***"            P3
...
else
  s := "too long, sorry :*("
```
- what is the relationship between $P_1$ and $P_0$?
    - $P_1 \equiv P_0$ `; s := s + "*"`
    - take the time to replace $P_1$ and $P_0$ in the formula above
- the same applies to all the other adjacent pairs
    - let us call $\Delta_P \equiv$ s := s + "*"
    - $P_2 \equiv P_1; \Delta_P$
    - $P_3 \equiv P_2; \Delta_P$
    - take a lot of time to replace each $P_{n+1}$, $P_n$, $\Delta_P$ in the formula above
- we could generalize this to say that for each step $n$, $P_{n+1} \equiv P_n; \Delta_P$
    - how many steps do we need? `l`, the input!
    - this means that, given an input `l`, the output program to run in order to produce the desired input is $P_0; \Delta_P^l$, that is $\underbrace{P_0; \Delta_P; \dots; \Delta_P}_{\text{l times}}$
- we introduce a new construct, the `while` loop, to assist us in expressign this construct
    - `while c do b` combines together two pieces of code, which it then coordinates together to perform a unique function
        - `c` is a boolean expression which determines whether or not we should stop right away; it is called **condition** of the loop
        - `b` is a full-blown sub-program which is run once every time a check on `c` is passed; it is called **body** of the loop
- the semantics of `while` make use of the semantics of `if`, given that they are already defined
    - `eval(<while False do b>, S) ` $\rightarrow$ `<done>, S`
    - `eval(<while c do b>, S) ` $\rightarrow$ `<if c then { b; while c do b } else {} >, S`
    - repetition is given by the second rule: a `while` performs first an `if`, and if the check succeeds then we run the sub-program `b` once, followed by the `while` yet again
- let us see the full traces of some example evaluations:
```
i := 0
while i < 3 do
  i := i + 1
```

```
i := 1
while i < 4 do
  i := i * 2
```

```
i := 1
while i < 6 do
  i := i + 1
  i := i * 2
```

- let us go to back the original example with named sub-programs

```
l := int(input())
if l <= 0 then
  s := ""               P0
elif l = 1 then
  s := "*"              P1
elif l = 2 then
  s := "**"             P2
elif l = 3 then
  s := "***"            P3
...
```

- as we said, each sub-program in the `if` statement can be replaced with $\D_P \equiv$ `s := s + "*"` repeated as many times as `l`, and starting from $P_0$
- this would lead us to an expanded wersion:

```
l := int(input())
if l <= 0 then
  P0
elif l = 1 then
  P0
  s := s + "*"
elif l = 2 then
  P0
  s := s + "*"
  s := s + "*"
elif l = 3 then
  P0
  s := s + "*"
  s := s + "*"
  s := s + "*"
...
```
- since all blocks start with a $P_0$, let us move it outside of the conditional 

```
l := int(input())
P0
if l <= 0 then

elif l = 1 then
  s := s + "*"
elif l = 2 then
  s := s + "*"
  s := s + "*"
elif l = 3 then
  s := s + "*"
  s := s + "*"
  s := s + "*"
...
```
- we can now re-encode this with a `while` loop:


```
l := int(input())
s := ""
while l > 0 do
  s := s + "*"
  l := l - 1
```

- notice that we are not only repeating $\Delta_P$, we are also decreasing `l` in order to count the number of times we needed to repeat $\Delta_P$
- let us run this program extensively a series of times, for different initial inputs:
    - $input := \{ next := "1", rest := \dots \}$
    - $input := \{ next := "2", rest := \dots \}$
    - $input := \{ next := "0", rest := \dots \}$
- how many paths inside the graph of all possible program traces are covered by the program above?
    - infinite, and of infinite length!
    - this is a significant improvement in expressive power

### Nesting of loops and conditionals
- the body of the loop is just a sub-program
- this means it can contain any statement imaginable
    - this includes conditionals
    - conditionals inside loops give us a lot of expressive power
    
```
l := int(input())
s := ""
while l > 0 do
  if l % 2 = 0 then
    s := s + "+"
  else
    s := s + "*"
  l := l + 1
```

- let us run this for a series of different inputs
    - $input := \{ next := "1", rest := \dots \}$
    - $input := \{ next := "2", rest := \dots \}$
    - $input := \{ next := "0", rest := \dots \}$
- in general, a conditional inside a loop yields a huge coverage inside the graph of all possible program traces

```
while c do
  if c' then
    p
  else
    q
```
    - covers all possible sequences of `p` and `q` statements
        - `p;p;p;p;...`
        - `q;q;q;q;...`
        - `p;q;p;q;...`
        - `p;p;q;q;...`
        - ...
    - each step of the loop makes a decision, leading to two possible paths
    - if the loop takes $l$ steps, then we get $2^l$ total possible paths
    - of course, the loop might take all possible numbers $l$ of steps, as far as we know
    - so this loop already covers a huge amount of paths
    
### Nesting of loops and loops
- the body of the loop can also contain other loops
    - this allows us to partition a problem requiring looping into multiple steps, each of which requires looping in turn
- for example, building a multiplication table (let us show the details of this program trace):

```
i := 2
s := ""
while i <= 4
  j := 1
  while j <= 3
    s := s + " " + str(i * j)
    j := j + 1
  s := s + "\n"
  i := i + 1
```

